In [ ]:
import os
import numpy as np
import rasterio
from PIL import Image
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import imageio.v2 as imageio
from pyproj import Transformer
import geopandas as gpd

In [ ]:
# File paths, loop or proceed one by one for all seasons
turbine_dir = "Input/Wind_Rotor"
wind_speed_path = "Input/Wind_Speed_Autumn.tif"
wind_dir_path = "Input/Wind_Sector_Autumn.tif"
output_video = "Output/Wind_Map_Autumn.mp4"
frame_dir = "Output/Frames_Autumn"
uk_path = "Input/Country_Borders/Borders_UK.shp"
ni_path = "Input/Country_Borders/Borders_NI.shp"
borders_path = "Input/Country_Borders/Line_Borders.shp"
os.makedirs(frame_dir, exist_ok=True)

In [ ]:
# Video settings
fps = 24
duration_sec = 8 # Customisation Line
n_frames = fps * duration_sec
step = 4 # Customisation Line
icon_scale = 0.5 # Customisation Line
icon_canvas_size = 200 

# Load wind speed and direction
with rasterio.open(wind_speed_path) as src:
    wind_speed = src.read(1)
    transform_affine = src.transform
    src_crs = src.crs
    bounds = src.bounds
    height, width = wind_speed.shape

with rasterio.open(wind_dir_path) as src:
    wind_dir = src.read(1)

# Transformer and projected extent
dst_crs = ccrs.epsg(3857)
transformer = Transformer.from_crs(src_crs, "EPSG:3857", always_xy=True)
lon_ul, lat_ul = transform_affine * (0.5, 0.5)
lon_lr, lat_lr = transform_affine * (width - 0.5, height - 0.5)
xmin, ymin = transformer.transform(lon_ul, lat_lr)
xmax, ymax = transformer.transform(lon_lr, lat_ul)

# Load turbine icons
turbine_icons = {
    0: Image.open(os.path.join(turbine_dir, "Wind_Rotor_N.png")),
    1: Image.open(os.path.join(turbine_dir, "Wind_Rotor_NE.png")),
    2: Image.open(os.path.join(turbine_dir, "Wind_Rotor_E.png")),
    3: Image.open(os.path.join(turbine_dir, "Wind_Rotor_SE.png")),
    4: Image.open(os.path.join(turbine_dir, "Wind_Rotor_S.png")),
    5: Image.open(os.path.join(turbine_dir, "Wind_Rotor_SW.png")),
    6: Image.open(os.path.join(turbine_dir, "Wind_Rotor_W.png")),
    7: Image.open(os.path.join(turbine_dir, "Wind_Rotor_NW.png")),
}

# Load and reproject country boundaries
uk_shape = gpd.read_file(uk_path).to_crs("EPSG:3857")
ni_shape = gpd.read_file(ni_path).to_crs("EPSG:3857")
borders_shape = gpd.read_file(borders_path).to_crs("EPSG:3857")

In [ ]:
# Generate frames
frames = []

for t in range(n_frames):
    print(f"Rendering frame {t+1}/{n_frames}")
    fig = plt.figure(figsize=(16, 20), dpi=300)
    fig.patch.set_facecolor('#324b5a') # Customisation Line
    fig.subplots_adjust(left=0.01, right=0.99, top=0.99, bottom=0.01) # Customisation Line

    ax = plt.axes(projection=dst_crs)
    ax.set_extent([xmin, xmax, ymin, ymax], crs=dst_crs)
    ax.set_axis_off()

    # Draw background polygons
    ax.add_geometries(uk_shape.geometry, crs=dst_crs, facecolor="#758b83", edgecolor="none", zorder=1) # Customisation Line
    ax.add_geometries(ni_shape.geometry, crs=dst_crs, facecolor="#617a70", edgecolor="none", zorder=2) # Customisation Line

    # Draw country borders
    ax.add_geometries(borders_shape.geometry, crs=dst_crs, facecolor="none", edgecolor="#617a70", linewidth=4, zorder=3) # Customisation Line

    fig.canvas.draw()
    trans = ax.transData.transform

    for row in range(0, height, step):
        for col in range(0, width, step):
            speed = wind_speed[row, col]
            direction_class = wind_dir[row, col]

            if np.isnan(speed) or np.isnan(direction_class):
                continue

            direction_class = int(direction_class)
            if direction_class not in turbine_icons:
                continue

            base_icon = turbine_icons[direction_class]

            canvas = Image.new("RGBA", (icon_canvas_size, icon_canvas_size), (0, 0, 0, 0))
            icon_x = (icon_canvas_size - base_icon.width) // 2
            icon_y = (icon_canvas_size - base_icon.height) // 2
            canvas.paste(base_icon, (icon_x, icon_y), mask=base_icon)

            angle = (np.log1p(speed) **2 * 10 * t + speed * 100) % 360  # Customisation Line
            
            rotated = canvas.rotate(angle, expand=False)

            size = int(icon_canvas_size * icon_scale)
            icon_final = rotated.resize((size, size), resample=Image.NEAREST)

            lon, lat = transform_affine * (col + 0.5, row + 0.5)
            x_proj, y_proj = transformer.transform(lon, lat)
            x_disp, y_disp = trans((x_proj, y_proj))

            x_img = int(x_disp - icon_final.width / 2)
            y_img = int(y_disp - icon_final.height / 2)
            fig.figimage(icon_final, xo=x_img, yo=y_img, zorder=10)

    frame_path = os.path.join(frame_dir, f"frame_{t:03d}.png")
    fig.savefig(frame_path, transparent=False, dpi=300)  # Use transparent=False to keep pink background
    plt.close()
    frames.append(imageio.imread(frame_path))

In [ ]:
# Save video
imageio.mimsave(output_video, frames, fps=fps, macro_block_size=1)
print(f"Video saved to {output_video}")